In [1]:
import pandas as pd
import numpy as np 

train = pd.read_csv('/Users/emilyvincett/Downloads/titanic/train.csv')

#Drop cabin from features as too many null values
features_to_use = ['PassengerId','Survived','Pclass','Sex','Age',
                   'SibSp','Parch','Fare','Embarked']

train = train[features_to_use]

#Drop null values in Age and Embarked
train = train.dropna()
y = train['Survived'].astype(str)
X = train.drop('Survived',axis=1).astype(str)



    

In [44]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import OneHotEncoder

# Split the data
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)
enc = OneHotEncoder(sparse=False)

#Encode the gender column on training set 
to_encode = X_train['Sex'].values.reshape(-1,1)
encoded_sex = enc.fit_transform(to_encode)
encoded_sex_df = pd.DataFrame(encoded_sex)
encoded_sex_df = encoded_sex_df.rename({0:'Female',1:'Male'},axis=1).reset_index().set_index(X_train.index).reset_index()

X_train = X_train.reset_index().merge(encoded_sex_df, left_on='index',right_on='level_0').set_index('index_x').drop(['Sex','level_0','index_y','blank'],axis=1)

#Encode the embarked column on training set 
to_encode = X_train['Embarked'].values.reshape(-1,1)
encoded_embark = enc.fit_transform(to_encode)
encoded_embark_df  = pd.DataFrame(encoded_embark).reset_index().set_index(X_train.index).drop('index',axis=1)
X_train = X_train.merge(encoded_embark_df,left_on='index_x',right_on='index_x').rename({0:'Cherbourg',1:'Queenstown',2:'Southampton'},axis=1)


#Encode the gender column in the test set
to_encode = X_test['Sex'].values.reshape(-1,1)
encoded_sex = enc.fit_transform(to_encode)
encoded_sex_df = pd.DataFrame(encoded_sex)
encoded_sex_df = encoded_sex_df.rename({0:'Female',1:'Male'},axis=1).reset_index().set_index(X_test.index).reset_index()

X_test = X_test.reset_index().merge(encoded_sex_df, left_on='index',right_on='level_0').set_index('index_x').drop(['Sex','level_0','index_y','blank'],axis=1)

#Encode the embarked column on training set 
to_encode = X_test['Embarked'].values.reshape(-1,1)
encoded_embark = enc.fit_transform(to_encode)
encoded_embark_df  = pd.DataFrame(encoded_embark).reset_index().set_index(X_test.index).drop('index',axis=1)
X_test = X_test.merge(encoded_embark_df,left_on='index_x',right_on='index_x').rename({0:'Cherbourg',1:'Queenstown',2:'Southampton'},axis=1)


X_train = X_train.drop('Embarked',axis=1)
X_test = X_test.drop('Embarked',axis=1)

In [52]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV


#Instantiate the classifier and save in variable
clf = RandomForestClassifier(random_state=0)

grid_values = {'n_estimators':[100,200],
              'criterion':['gini','entropy'],
              'max_features':['auto','sqrt','log2'],
              'class_weight':['balanced','balanced_subsample'],
              'max_depth':[10,20,50],
              'min_samples_split':[2,10,30]}

#Fit the classifier to GridSearch
grid_clf_auc = GridSearchCV(clf, param_grid=grid_values, n_jobs=-1,scoring='roc_auc')
grid_clf_auc.fit(X_train,y_train)
predicted_labels = grid_clf_auc.predict(X_test)

print('Model AUC %.3f' %(roc_auc_score(y_test,predicted_labels)))
print('Model Best Score %.3f' %(grid_clf_auc.best_score_))
print('Model Best Parameters {}'.format(grid_clf_auc.best_params_))





Model AUC 0.800
Model Best Score 0.877
Model Best Parameters {'class_weight': 'balanced_subsample', 'criterion': 'entropy', 'max_depth': 10, 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 100}


In [ ]:
# Radius of the earth in km 
r = 6371

# Take a specific hotel as a input
hotel = input('Enter Hotel ID ')
hotel = int(hotel)

# Match it to entry in the dataframe and pull out is long & lat
df.iloc[hotel]
long_h = radians(df.iloc[hotel]['longitude'])
lat_h = radians(df.iloc[hotel]['latitude'])

# Iterate over the dataframe for entries other than the input hotel to pull out their long and lat
for i in range(len(df)):
    if i != hotel: 
        long_i = radians(df.iloc[i]['longitude'])
        lat_i = radians(df.iloc[i]['latitude'])
        long_diff = long_i - long_h 
        lat_diff = lat_i - lat_h
        
        #Haversine formula to calculate great circle distance between two points 
        a = sin(lat_diff/2)**2 + cos(lat_h) * cos(lat_i) * sin(long_diff/2)**2 
        c = 2**asin(sqrt(a))
        
        # Create new column of distances from the input hotel
        df['distance'] = c*r
        # Sort values from nearest to furthest from input and pull out N closest hotels
        recommendations = df.sort_values('distance', ascencind=True)[:N]
        
        